# Створення додатків для генерації тексту

Ви вже бачили протягом цього курсу, що існують основні концепції, такі як запити, і навіть ціла дисципліна, яка називається "інженерія запитів". Багато інструментів, з якими ви можете взаємодіяти, як-от ChatGPT, Office 365, Microsoft Power Platform тощо, підтримують використання запитів для досягнення мети.

Щоб додати такий досвід у додаток, вам потрібно розуміти такі концепції, як запити, доповнення, та вибрати бібліотеку для роботи. Саме це ви й вивчите в цьому розділі.

## Вступ

У цьому розділі ви:

- Дізнаєтеся про бібліотеку openai та її основні концепції.
- Створите додаток для генерації тексту за допомогою openai.
- Зрозумієте, як використовувати такі концепції, як запит, температура та токени для створення додатку генерації тексту.є


## Вправа - створення генератора рецептів

Тепер, коли ми розглянули сценарій, давайте напишемо код, який відповідає продемонстрованому сценарію. Для цього виконайте наступні кроки:

1. Використовуйте існуючий файл як початкову точку
1. Створіть змінну `prompt` і змініть зразок коду, як показано нижче:

In [1]:
# practical_ai_recipe_generator.py
# ВПРАВА: Генератор рецептів із використанням LLM (GPT-4o через Azure AI Inference)

import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# 1️⃣ Ініціалізація ключа доступу
# Твій токен повинен бути збережений у системній змінній середовища
# наприклад: setx GITHUB_TOKEN "твій_токен"
token = os.environ["GITHUB_TOKEN"]

# 2️⃣ Налаштування клієнта для підключення до API
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# 3️⃣ Основний промпт (користувацький запит)
ingredients = "курка, картопля та морква"

prompt = f"""
Покажи мені 5 рецептів страви з наступними інгредієнтами: {ingredients}.
Для кожного рецепту перелічи всі використовувані інгредієнти.
"""

# 4️⃣ Створення запиту до моделі
response = client.complete(
    messages=[
        {
            "role": "system",
            "content": "Ти кулінарний асистент, який створює рецепти з урахуванням доступних інгредієнтів користувача.",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ],
    model=model_name,
    temperature=1.0,   # Рівень креативності
    max_tokens=1000,   # Максимальна довжина відповіді
    top_p=1.0
)

# 5️⃣ Виведення результату
print("=== РЕЦЕПТИ НА ОСНОВІ ІНГРЕДІЄНТІВ ===\n")
print(response.choices[0].message.content)

# 6️⃣ Додатковий запит — фільтрація та список покупок
follow_up_prompt = """
Будь ласка, видали рецепти з часником, оскільки у мене на нього алергія.
Замініть його чимось іншим (наприклад, цибулею або зеленню).
Також створіть список покупок для рецептів, враховуючи, що у мене вже є курка, картопля та морква вдома.
"""

response2 = client.complete(
    messages=[
        {
            "role": "system",
            "content": "Ти кулінарний асистент, який адаптує рецепти до обмежень користувача.",
        },
        {
            "role": "user",
            "content": follow_up_prompt,
        },
    ],
    model=model_name,
    temperature=0.9,
    max_tokens=1000
)

print("\n=== ОНОВЛЕНІ РЕЦЕПТИ БЕЗ АЛЕРГЕНІВ ===\n")
print(response2.choices[0].message.content)


=== РЕЦЕПТИ НА ОСНОВІ ІНГРЕДІЄНТІВ ===

Ось п’ять рецептів зі списком інгредієнтів для приготування страв із куркою, картоплею та морквою:

---

### 1. **Тушкована курка з картоплею та морквою**
**Інгредієнти:**
- Курка (400–500 г, стегна, гомілки чи філе)
- Картопля (5 шт.)
- Морква (2 шт.)
- Цибуля (1 шт.)
- Часник (2 зубчики)
- Вода або бульйон (200–300 мл)
- Сіль, перець (за смаком)
- Лавровий лист (1 шт.)
- Рослинна олія (2 ст. л.)
- Свіжа зелень (для подачі)

---

### 2. **Запечена курка з картоплею та морквою**
**Інгредієнти:**
- Курка (ціла, стегна, або філе – 1 кг)
- Картопля (6 шт.)
- Морква (3 шт.)
- Часник (3 зубчики)
- Оливкова олія (3 ст. л.)
- Паприка, сушений часник, куркума (по 1 ч. л.)
- Орегано або базилік (1 ч. л.)
- Сіль, перець (за смаком)
- Лимонний сік (опційно, 1 ст. л.)

---

### 3. **Крем-суп із курки, картоплі та моркви**
**Інгредієнти:**
- Куряче філе (300 г)
- Картопля (4 шт.)
- Морква (2 шт.)
- Цибуля (1 шт.)
- Вершки (200 мл, 10%-20% жирності)
- Вода або

1. Змінимо код наступним чином:

In [1]:
# practical_ai_recipe_generator_fixed.py
# ✅ Робоча версія генератора рецептів із GPT-4o через Azure AI Inference

import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# 1️⃣ Отримуємо токен
token = os.environ.get("GITHUB_TOKEN")
if not token:
    raise RuntimeError("Помилка: не знайдено змінну середовища GITHUB_TOKEN")

# 2️⃣ Налаштування клієнта
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# 3️⃣ Отримання параметрів від користувача
raw_n = input("Кількість рецептів (наприклад, 5): ").strip()
try:
    no_recipes = int(raw_n) if raw_n else 5
except ValueError:
    no_recipes = 5

ingredients = input("Введи інгредієнти (наприклад, курка, картопля, морква): ").strip()
if not ingredients:
    ingredients = "курка, картопля та морква"

# 4️⃣ Формування запиту
prompt = (
    f"Покажи мені {no_recipes} рецептів із такими інгредієнтами: {ingredients}. "
    "Для кожного рецепту перелічи всі інгредієнти, короткий опис приготування "
    "та орієнтовний час приготування у хвилинах."
)

# 5️⃣ Виклик моделі
response = client.complete(
    model=model_name,
    messages=[
        SystemMessage(content="Ти кулінарний асистент, який створює рецепти з урахуванням заданих інгредієнтів."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    max_tokens=1200,
    top_p=1.0
)

# 6️⃣ Безпечний вивід
print("\n=== ЗГЕНЕРОВАНІ РЕЦЕПТИ ===\n")

text = None
try:
    # стандартна структура для поточної версії SDK
    text = response.choices[0].message.content
except Exception:
    try:
        # fallback для нових версій SDK
        text = response.output_message.content[0].text
    except Exception:
        text = str(response)

print(text)

# 7️⃣ Додатковий етап – фільтрація рецептів
follow_up = input("\nХочеш змінити умови (наприклад, 'без часнику' або натисни Enter, щоб пропустити): ").strip()
if follow_up:
    response2 = client.complete(
        model=model_name,
        messages=[
            SystemMessage(content="Ти кулінарний асистент, який адаптує рецепти до обмежень користувача."),
            UserMessage(content=follow_up),
        ],
        temperature=0.9,
        max_tokens=1000
    )

    print("\n=== ОНОВЛЕНІ РЕЦЕПТИ ===\n")
    try:
        print(response2.choices[0].message.content)
    except Exception:
        try:
            print(response2.output_message.content[0].text)
        except Exception:
            print(response2)
else:
    print("\n(Фільтрацію пропущено)")



=== ЗГЕНЕРОВАНІ РЕЦЕПТИ ===

Ось п'ять рецептів із зазначеними інгредієнтами (курка, картопля, морква, ананас, кукурудза):

---

### 1. **Запечена курка з картоплею, морквою та ананасом**
**Інгредієнти:**
- Курка (грудинка або стегенця) — 500 г  
- Картопля — 500 г  
- Морква — 2 шт.  
- Консервований ананас (кільця) — 150 г  
- Олія — 2 ст. л.  
- Сіль, перець, спеції для курки — за смаком.

**Опис приготування:**
1. Почистіть картоплю та моркву, наріжте великими шматочками.
2. Змішайте овочі в мисці з олією та спеціями.
3. Курку приправте сіллю, перцем та улюбленими спеціями.
4. Викладіть овочі та курку на деко, зверху покладіть шматочки ананасу.
5. Запікайте в духовці при 180°C близько 40 хвилин, доки овочі пом'якшаться, а курка не підрум'яниться.

**Час приготування:** 50 хвилин.

---

### 2. **Курячий салат з ананасом та кукурудзою**
**Інгредієнти:**
- Відварене куряче філе — 300 г  
- Консервований ананас — 150 г  
- Консервована кукурудза — 150 г  
- Варена морква — 1 шт.  
- В

## Індивідуальне завдання

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# ✅ Додаток для аналізу дампів пам'яті x86/ARM процесорів


# 1️⃣ Ініціалізація токена
token = os.environ.get("GITHUB_TOKEN")
if not token:
    raise RuntimeError("Не знайдено системну змінну середовища GITHUB_TOKEN")

endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# 2️⃣ Ввід параметрів від користувача
processor_arch = input("Виберіть архітектуру процесора (x86/ARM): ").strip().upper()
if processor_arch not in ["X86", "ARM"]:
    processor_arch = "x86"

dump_type = input("Тип дампу пам'яті (stack trace/register dump/memory snapshot): ").strip().lower()
if not dump_type:
    dump_type = "stack trace"

# 3️⃣ Приклад дампу пам'яті для демонстрації
if processor_arch == "X86":
    sample_dump = """
    EAX: 0x12345678  EBX: 0x87654321
    ECX: 0xDEADBEEF  EDX: 0xCAFEBABE
    ESI: 0x11223344  EDI: 0x44332211
    EBP: 0x7FFFFFFF  ESP: 0x7FFFFFFC
    EIP: 0x08048000
    Stack: 0x7FFFFFFC: 0x08048100, 0x08048200, 0xFFFFFFFF
    """
else:
    sample_dump = """
    R0: 0x12345678   R1: 0x87654321
    R2: 0xDEADBEEF   R3: 0xCAFEBABE
    R4: 0x11223344   R5: 0x44332211
    R6: 0xABCDEF00   R7: 0x00FEDCBA
    R12: 0x7FFFFFFF  SP: 0x7FFFFFFC
    LR: 0x08048000   PC: 0x08048100
    """

print(f"\n📋 Приклад дампу пам'яті ({processor_arch}):\n{sample_dump}\n")

# 4️⃣ Формування запиту до моделі
prompt = (
    f"Проаналізуй наступний дамп пам'яті {processor_arch} процесора (тип: {dump_type}):\n\n"
    f"{sample_dump}\n\n"
    "Виконай такий аналіз:\n"
    "1) Визнач значення реєстрів та їх можливе призначення.\n"
    "2) Розкрий адреси стеку та проаналізуй послідовність виклику функцій.\n"
    "3) Вияви потенційні помилки (переповнення стеку, невідповідні значення реєстрів).\n"
    "4) Рекомендуй способи відлагодження проблеми.\n"
    "5) Поясни, як ця інформація допоможе у розробці або відлагодженні програми."
)

# 5️⃣ Виклик моделі GPT-4o
response = client.complete(
    messages=[
        {
            "role": "system",
            "content": "Ти експерт з низькорівневого програмування та анасу дампів пам'яті. Надай детальний технічний аналіз.",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ],
    model=model_name,
    temperature=0.8,
    max_tokens=1500,
    top_p=1.0
)

# 6️⃣ Виведення результату
print("=== АНАЛІЗ ДАМПУ ПАМ'ЯТІ ===\n")
print(response.choices[0].message.content)

# 7️⃣ Додатковий запит для рекомендацій
follow_up_prompt = input("\nЧи хочеш отримати детальніші рекомендації? (введи питання або натисни Enter, щоб пропустити): ").strip()
if follow_up_prompt:
    response2 = client.complete(
        messages=[
            {
                "role": "system",
                "content": "Ти експерт з низькорівневого програмування.",
            },
            {
                "role": "user",
                "content": follow_up_prompt,
            },
        ],
        model=model_name,
        temperature=0.7,
        max_tokens=1000
    )
    print("\n=== ДОДАТКОВІ РЕКОМЕНДАЦІЇ ===\n")
    print(response2.choices[0].message.content)
else:
    print("\n(Додатковий запит пропущено)")